###### **Problema de Transportes**
Problema retidado do livro Pesquisa Operacional

O problema consiste em definir a quantidade de um produto transportada de 2 centros de produção para 3 mercados consumidores.

O centro de produção 1 possui uma capacidade de 800 unidades, enquanto o centro 2 possui uma capacidade de 1000. Estas capacidades não podem ser excedidas.

A demandas dos mercados 1, 2 e 3 são 500, 400 e 900, respectivamente.

Existe um custo unitário de transporte de cada centro para cada mercado.

O objetivo do problema é encontrar a solução de menor custo que atenda toda a demanda

###### **O modelo**

Variável
*   $x_{ij}$: Quantidade transportada do centro $i$ para o mercado $j$

Função objetiva

$\min \quad 4x_{11}+2x_{12}+5x_{13}+11x_{21}+7x_{22}+4x_{23}$

Restrições de capacidade dos centros

$x_{11} + x_{12} + x_{13} \leq 800$

$x_{21} + x_{22} + x_{23} \leq 1000$

Restrições de atendimento de demanda

$x_{11} + x_{21} = 500$

$x_{12} + x_{22} = 400$

$x_{13} + x_{23} = 900$

Restrições de não-negatividade

$x_{11},x_{12},x_{13},x_{21},x_{22},x_{23} \geq 0$


In [4]:
%pip install -i https://pypi.gurobi.com gurobipy
import gurobipy as gp
from gurobipy import GRB

Looking in indexes: https://pypi.gurobi.com
Note: you may need to restart the kernel to use updated packages.


In [5]:
m = gp.Model("Transportes")

x_11 = m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="Transporte_1_1")
x_12 = m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="Transporte_1_2")
x_13 = m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="Transporte_1_3")
x_21 = m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="Transporte_2_1")
x_22 = m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="Transporte_2_2")
x_23 = m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="Transporte_2_3")

m.setObjective(4*x_11 + 2*x_12 + 5*x_13 +
               11*x_21 + 7*x_22 + 4*x_23,
               GRB.MINIMIZE)

m.addConstr(x_11 + x_12 + x_13 <= 800,
            name="Capacidade Centro 1")
m.addConstr(x_21 + x_22 + x_23 <= 1000,
            name="Capacidade Centro 2")
m.addConstr(x_11 + x_21 == 500, name="Demanda Mercado 1")
m.addConstr(x_12 + x_22 == 400, name="Demanda Mercado 2")
m.addConstr(x_13 + x_23 == 900, name="Demanda Mercado 3")

m.optimize()


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xfbd52b1b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 1e+03]
Presolve removed 4 rows and 3 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.4000000e+03   1.250000e+01   0.000000e+00      0s
       1    6.9000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  6.900000000e+03


In [6]:
print("Custo mínimo: "+str(m.objVal))
print("Quantidade enviada:")
print("Centro\tMercado\tQuantidade")
print("1\t1\t"+str(x_11.X))
print("1\t2\t"+str(x_12.X))
print("1\t3\t"+str(x_13.X))
print("2\t1\t"+str(x_21.X))
print("2\t2\t"+str(x_22.X))
print("2\t3\t"+str(x_23.X))

Custo mínimo: 6900.0
Quantidade enviada:
Centro	Mercado	Quantidade
1	1	500.0
1	2	300.0
1	3	0.0
2	1	0.0
2	2	100.0
2	3	900.0


###### **O modelo generalizado**

Parâmetros
*   $m$: Número de centros de produção
*   $n$: Número de mercados consumidores
*   $a_i$: Capacidade do centro $i$
*   $b_j$: Demanda do mercado $j$
*   $c_{ij}$: Custo de transporte de uma unidade do centro $i$ para o mercado $j$

Variável
*   $x_{ij}$: Quantidade transportada do centro $i$ para o mercado $j$

Função objetiva

$\min \sum_{i=1}^m \sum_{j=1}^n c_{ij}x_{ij}$

Restrições de capacidade dos centros

$\sum_{j=1}^n x_{ij} \leq a_i \quad i = 1,\dots,m$

Restrições de atendimento de demanda

$\sum_{i=1}^m x_{ij} = b_j \quad j = 1,\dots,n$

Restrições de não-negatividade

$x_{ij} \geq 0 \quad i = 1,\dots,m \quad j = 1,\dots,n$

In [7]:
M = 2
N = 3

a = [800,1000]
b = [500,400,900]
c = [[4,2,5],
    [11,7,4]]

m = gp.Model("Transportes_2")

x = m.addVars(M,N, vtype=GRB.CONTINUOUS, lb=0, name="Transporte")

m.setObjective(sum(c[i][j]*x[i,j] for i in range(M) for j in range(N)),
               GRB.MINIMIZE)

m.addConstrs((gp.quicksum(x[i,j] for j in range(N)) <= a[i] for i in range(M)),
             name="Capacidade Centros")
m.addConstrs((gp.quicksum(x[i,j] for i in range(M)) == b[j]  for j in range(N)),
             name="Demanda Mercados")

m.optimize()


Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xfbd52b1b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 1e+03]
Presolve removed 4 rows and 3 columns
Presolve time: 0.00s
Presolved: 1 rows, 3 columns, 3 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.4000000e+03   1.250000e+01   0.000000e+00      0s
       1    6.9000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  6.900000000e+03


**Problema de designação**

Quatro edifícios diferentes A, B, C e D devem ser construidos em um campus universitário por quatro empreiteiras 1, 2, 3 e 4.

Como todas as empreiteiras contribuem muito para o fundo dos alunos,
cada uma delas deve construir um edifício.

Cada empreiteira fez suas propostas no tocante às quatro construções. O problema consiste em determinar que construção designar a que empreiteira para que o custo da obtenção dos quatro edifícios permaneça mínimo.

Os custos são os mesmos apresentados nos slides da aula anterior.

**O modelo generalizado**

Parâmetros
*   $C$: Número de construções
*   $E$: Número de empreiteiras
*   $c_{ij}$: Custo da construção $i$ com a empreiteira $j$

Variável
*   $x_{ij}$: 1 se a construção $i$ é feita com a empreiteira $j$, 0 caso contrário

Função objetiva

$\min \sum_{i=1}^C \sum_{j=1}^E c_{ij}x_{ij}$

Restrições de designação de 1 empreiteira por construção

$\sum_{j=1}^E x_{ij} = 1 \quad i = 1,\dots,C$

Restrições de designação de 1 construção por empreiteira

$\sum_{i=1}^C x_{ij} = 1 \quad j = 1,\dots,E$

Restrições de domínio

$x_{ij} = \{0,1\} \quad i = 1,\dots,m \quad j = 1,\dots,n$


In [8]:
C = 4
E = 4

c = c = [[48,48,50,44],
    [56,60,60,68],
    [96,94,90,85],
    [42,44,54,46]]

m = gp.Model("Designação")

x = m.addVars(C,E, vtype=GRB.BINARY)

m.setObjective(sum(c[i][j]*x[i,j] for i in range(C) for j in range(E)),
               GRB.MINIMIZE)

m.addConstrs((gp.quicksum(x[i,j] for j in range(E)) == 1 for i in range(C)))
m.addConstrs((gp.quicksum(x[i,j] for i in range(C)) == 1  for j in range(E)))

m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 16 columns and 32 nonzeros
Model fingerprint: 0x9424b02a
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 246.0000000
Presolve time: 0.00s
Presolved: 8 rows, 16 columns, 32 nonzeros
Variable types: 0 continuous, 16 integer (16 binary)

Root relaxation: objective 2.340000e+02, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     234.0000000  234.00000  0.00%     -    0s

Explored 1 nodes (7 simplex iterations) in 0.01 seconds (0.0

In [9]:
contr = ["A","B","C","D"]

print("Custo mínimo: "+str(m.objVal))

print("\t\tEmpreiteira")
print("Constr.\t1\t2\t3\t4")

for i in range(C):
  print(contr[i], end = '')
  for j in range(E):
    print("\t"+str(round(x[i,j].X)), end = '')
  print()


Custo mínimo: 234.0
		Empreiteira
Constr.	1	2	3	4
A	0	0	0	1
B	1	0	0	0
C	0	0	1	0
D	0	1	0	0
